# Running SCellBOW phenotype algebra on a simulated prostate cancer dataset
> ### There are three molecular subtype - ARAH, ARAL, and NE

In [1]:
import SCellBOW as sb
import scanpy as sc
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /home/namratab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# SCellBOW Pretrained model

In [2]:
adata_source = sc.read("data/adata_source.h5ad")
adata_source

AnnData object with n_obs × n_vars = 1334 × 2000

In [3]:
adata_train = sc.read("data/adata_train.h5ad")
adata_train

/home/namratab/anaconda3/envs/namrata/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 81 × 17981
    obs: 'subtype', 'time', 'status'
    var: 'n_cells'

In [4]:
adata_target = sc.read("data/adata_target.h5ad")
adata_target

AnnData object with n_obs × n_vars = 836 × 2000
    obs: 'subtype'

In [5]:
# preprocessing source data
adata_source.var_names_make_unique()
sc.pp.filter_cells(adata_source, min_genes=10)
sc.pp.filter_genes(adata_source, min_cells=2)

sc.pp.normalize_total(adata_source, target_sum=1e4)
sc.pp.log1p(adata_source)
    
sc.pp.highly_variable_genes(adata_source, n_top_genes = 1000)
adata_source = adata_source[:, adata_source.var.highly_variable]

sc.pp.scale(adata_source, max_value=10)
adata_source

/home/namratab/anaconda3/envs/namrata/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


AnnData object with n_obs × n_vars = 1334 × 1000
    obs: 'n_genes'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'log1p', 'hvg'

In [6]:
#invoke SCellBOW pretraining
sb.SCellBOW_pretrain(adata_source, save_dir= "pca", vec_size=300, n_worker=1, iter=20)

[ 2022-12-30 11:12:51.743037 ] The path to save directory is ./pca/
[ 2022-12-30 11:12:51.743071 ] Creating the source model.
[ 2022-12-30 11:12:51.753967 ] Creating the corpus.


  0%|          | 0/1334 [00:00<?, ?it/s]

[ 2022-12-30 11:12:52.537066 ] Corpus created with size = 1334
[ 2022-12-30 11:12:52.537092 ] Tagging the corpora.
[ 2022-12-30 11:12:55.919765 ] All corpuses tagged with length 1334
[ 2022-12-30 11:12:55.919830 ] Inititalize the SCellBOW source model.
[ 2022-12-30 11:12:55.919840 ] INFO - SCellBOW: vector size = 300
[ 2022-12-30 11:12:55.919849 ] INFO - SCellBOW: initial learning rate = 0.025
[ 2022-12-30 11:12:55.919862 ] INFO - SCellBOW: min_alpha = 0.00025
[ 2022-12-30 11:12:55.919872 ] INFO - SCellBOW: min_count = 1
[ 2022-12-30 11:12:55.919881 ] INFO - SCellBOW: number of cpu = 1
[ 2022-12-30 11:12:55.920217 ] Building vocabulary.
[ 2022-12-30 11:12:56.080118 ] Vocabulary built.
[ 2022-12-30 11:12:56.080141 ] Start training the neural network.
[ 2022-12-30 11:13:34.161856 ] Training SCellBOW source model finished.
[ 2022-12-30 11:13:34.168458 ] Model saved in directory  ./pca/
[ 2022-12-30 11:13:34.185962 ] Source model created!


# SCellBOW Phenotype algebra

In [ ]:
# invoke SCellBOW phenotype algebra on "subtype"
median_score, predicted_score=sb.SCellBOW_algebra(adata_target,adata_train,"pca", type='subtype',
                    algebra = ['ARAH','ARAL'],
                    bootstrap_samples=10,n_top_features=1000).run()

[ 2022-12-30 11:13:34.199583 ] The path to save directory is pca/
[ 2022-12-30 11:13:34.199696 ] Begin SCellBOW: phenotype algebra.
[ 2022-12-30 11:13:34.362316 ] Begin creating pseudobulk on subtype


/home/namratab/anaconda3/envs/namrata/lib/python3.8/site-packages/SCellBOW/SCellBOW.py:308: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe['type'] = y


[ 2022-12-30 11:13:34.564571 ] Shape of pseudobulk by individual type: (3, 1999)
[ 2022-12-30 11:13:34.656679 ] Adding combined vector for algebra: ['ARAH', 'ARAL']
[2022-12-30 11:13:34.700492]Common gene between survival and target data=675
[ 2022-12-30 11:13:34.720032 ] Begin transfer learning.
[ 2022-12-30 11:13:34.728370 ] The path to save directory is ./pca//
[ 2022-12-30 11:13:34.728385 ] Begin SCellBOW: transfer learning.
MinMaxScaler(feature_range=(1, 10))
[ 2022-12-30 11:13:34.734667 ] Creating the corpus.


/home/namratab/anaconda3/envs/namrata/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/namratab/anaconda3/envs/namrata/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


  0%|          | 0/86 [00:00<?, ?it/s]

[ 2022-12-30 11:13:34.838856 ] Tagging the corpora for transfer learning.
[ 2022-12-30 11:13:35.303428 ] All corpuses tagged with length = 86
[ 2022-12-30 11:13:35.303482 ] Updating the vocabulary.
[ 2022-12-30 11:13:35.341712 ] Vocabulary updated.
[ 2022-12-30 11:13:35.341733 ] Start transfer learning on the neural network.
[ 2022-12-30 11:13:47.824727 ] Weights of the neural network calibrated.
[ 2022-12-30 11:13:47.824806 ] Start infering the vectors for target dataset.


  0%|          | 0/86 [00:00<?, ?it/s]

In [ ]:
predicted_score

In [ ]:
median_score

In [ ]:
#visualisation
median_score.sort_values(ascending=True, inplace=True)
predicted_score = predicted_score[median_score.index]
plt.figure(figsize=(4,4))
predicted_score.boxplot(patch_artist=True, notch=True)
plt.xticks(rotation=90, size=10)
plt.show()